In [9]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
Basic_blocks = drive.CreateFile({'id':'1LIJahCBOgKswPjV3dCdjno6iQunL1JQN'})
FusionNet = drive.CreateFile({'id':'1tEd6iRLRH-aQO7dzZkkYnZ4sYDVSaNNz'})
UNet = drive.CreateFile({'id':'1Xf0xSo-CswXiuiUR_tc9IbvzRx-UWU7M'})

In [0]:
Basic_blocks.GetContentFile('Basic_blocks.py')
FusionNet.GetContentFile('FusionNet.py')
UNet.GetContentFile('UNet.py')

In [0]:
from FusionNet import *
from UNet import *
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# hyperparameters
batch_size = 2
img_size = 512
lr = 0.0005
epoch = 10

In [0]:
# input pipeline
img_dir = "/content/drive/My Drive/.../Datasets/DeepGlobe/Roads"

img_data = dset.ImageFolder(root=img_dir, transform=transforms.Compose([
    transforms.Resize(size=img_size),
    transforms.CenterCrop(size=(img_size, img_size * 2)),
    transforms.ToTensor(),
]))
img_batch = data.DataLoader(img_data, batch_size=batch_size,
                            shuffle=True, num_workers=0)

In [16]:
# initiate Generator
generator = UnetGenerator(3, 3, 64).cuda()


------Initiating U-Net------



In [17]:
# load pretrained model

path = "/content/drive/My Drive/.../Datasets/DeepGlobe/Roads/Unet.pkl"

try:
    generator = torch.load(path)
    print("\n--------model restored--------\n")
except:
    print("\n--------model not restored--------\n")
    pass



--------model restored--------



In [0]:
savepath = "content/drive/My Drive/.../Datasets/DeepGlobe/Roads/result/"
  
# loss function 

loss_func = nn.MSELoss()
gen_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)

# training

#file = open('./content/drive/My Drive/Major Project/Images/Datasets/DeepGlobe/Roads/Unet_mse_loss', 'w')
for i in range(epoch):
    for _, (image, label) in enumerate(img_batch):
        satel_image, map_image = torch.chunk(image, chunks=2, dim=3)

        gen_optimizer.zero_grad()

        x = Variable(satel_image).cuda(0)
        y_ = Variable(map_image).cuda(0)
        y = generator.forward(x)

        loss = loss_func(y, y_)
        #file.write(str(loss) + "\n")
        loss.backward()
        gen_optimizer.step()

        if _ % 400 == 0:
            print(i)
            print(loss)
            #v_utils.save_image(x.cpu().data, savepath+"original_image_{}_{}.png".format(i, _))
            #v_utils.save_image(y_.cpu().data, savepath+"label_image_{}_{}.png".format(i, _))
            #v_utils.save_image(y.cpu().data, savepath+"gen_image_{}_{}.png".format(i, _))
            torch.save(generator, path)

0
Variable containing:
1.00000e-02 *
  2.3387
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0
Variable containing:
1.00000e-02 *
  1.9865
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0
Variable containing:
1.00000e-03 *
  8.2340
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0
Variable containing:
1.00000e-02 *
  1.0552
[torch.cuda.FloatTensor of size 1 (GPU 0)]

